In [ ]:
'''mkdir Project_work
cd Project_work
mkdir Pdf_docs
copy the pdf files here
copy the provided 'sales_data.csv' here

--create virtual environment
python -m venv venv
cd  venv/Scripts
activate
(venv) C:\Users\AJ\Downloads\Project_work\venv>

--in virtual env install required packages
pip install pandas
python -m pip install --upgrade pip
pip install pypdf
pip install transformers
pip install numpy
pip install scipy
pip install openai
pip install langchain_community
pip install langchain-core
pip install langchain
'''

In [ ]:
#Project structure
#package to work with excel
#create df which contains from excel
#analysis = summary
#work with pdf documents >load > split > embedd > storing them in RAG layer
#langchain >
  #llm definition (myllm)
  #prompt template > scenario to form a prompt
  #prompt = PromptTemplate(template=scenario,input_variables=["analysis","question"])
#my_proj_chain = LLMChain(prompt=prompt,llm=myllm)
#my_proj_chain.run(analysis,question)
#question = 'based on my analysis on data, what strategies are recommeded for improvment of sales'
#my_proj_chain.run(analysis,question)

##implementation of seq chain to do the same thing as above in 2 different chains followed
#sequentially

#1st
#data_analysis_template = """analyze my {analysis} data  and provide a summary """
#a_prompt = PromptTemplate(template=data_analysis_template,input_variables=["analysis"])
#chain1 = LLMChain(prompt=a_prompt,llm=myllm,output='output')

#2nd
#recommendation_template = """based on {analysis} data  and provide a recommendaton to address the que """
#b_prompt = PromptTemplate(template=recommendation_template,input_variables=["analysis","question"])
#chain2 = LLMChain(prompt=b_prompt,llm=myllm,output='output2')

"""overall_chain = SequentialChain(
       chains = [chain1,chain2],
       input_variables=[],
       output_variables = [],
)

run the chain for a particular question"""
#------------

#Bulding RAG based layer
#once all data is chunked, embedded and pushed into FAISS vector store..
#llm = ChatOpenAI(temperature=0.7,model_name='gpt-4.1-mini')
"""retriever = vectorstore.as_retriever(search_type="similarity",sarch_kwags={"k": 2})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever =retriever,
    return_source_documents=True
)"""

#implementation of a tool like wikipedia search

#implemenation of usge of memory to benefit response generation

#integrate of evaluation
#answers to validate responses





In [2]:
#!pip install pandas
import pandas as pd

In [4]:
mydata = pd.read_csv('/content/sales_data.csv')

In [5]:
mydata

,Date,Product,Region,Sales,Customer_Age,Customer_Gender,Customer_Satisfaction
0,1/1/2022,Widget C,South,786,26,Male,2.874407
1,1/2/2022,Widget D,East,850,29,Male,3.365205
2,1/3/2022,Widget A,North,871,40,Female,4.547364
3,1/4/2022,Widget C,South,464,31,Male,4.555420
4,1/5/2022,Widget C,South,262,50,Female,3.982935
...,...,...,...,...,...,...,...
2495,10/31/2028,Widget D,North,979,57,Male,3.525510
2496,11/1/2028,Widget D,South,858,30,Female,3.386064
2497,11/2/2028,Widget B,East,878,21,Female,2.272609
2498,11/3/2028,Widget C,South,862,63,Male,2.805692


In [6]:
mydata.describe

<bound method NDFrame.describe of             Date   Product Region  Sales  Customer_Age Customer_Gender  \
0       1/1/2022  Widget C  South    786            26            Male   
1       1/2/2022  Widget D   East    850            29            Male   
2       1/3/2022  Widget A  North    871            40          Female   
3       1/4/2022  Widget C  South    464            31            Male   
4       1/5/2022  Widget C  South    262            50          Female   
...          ...       ...    ...    ...           ...             ...   
2495  10/31/2028  Widget D  North    979            57            Male   
2496   11/1/2028  Widget D  South    858            30          Female   
2497   11/2/2028  Widget B   East    878            21          Female   
2498   11/3/2028  Widget C  South    862            63            Male   
2499   11/4/2028  Widget D   West    614            31            Male   

      Customer_Satisfaction  
0                  2.874407  
1                

In [7]:
def generate_advanced_data_summary(df):
    # Ensure 'Date' is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Sales Analysis
    total_sales = df['Sales'].sum()
    avg_sale = df['Sales'].mean()
    median_sale = df['Sales'].median()
    sales_std = df['Sales'].std()

    # Time-based Analysis
    df['Month'] = df['Date'].dt.to_period('M')
    monthly_sales = df.groupby('Month', observed=False)['Sales'].sum().sort_values(ascending=False)
    best_month = monthly_sales.index[0]
    worst_month = monthly_sales.index[-1]

    # Product Analysis
    product_sales = df.groupby('Product', observed=False)['Sales'].agg(['sum', 'count', 'mean'])
    top_product = product_sales['sum'].idxmax()
    most_sold_product = product_sales['count'].idxmax()

    # Regional Analysis
    region_sales = df.groupby('Region', observed=False)['Sales'].sum().sort_values(ascending=False)
    best_region = region_sales.index[0]
    worst_region = region_sales.index[-1]

    # Customer Analysis
    avg_satisfaction = df['Customer_Satisfaction'].mean()
    satisfaction_std = df['Customer_Satisfaction'].std()

    age_bins = [0, 25, 35, 45, 55, 100]
    age_labels = ['18-25', '26-35', '36-45', '46-55', '55+']
    df['Age_Group'] = pd.cut(df['Customer_Age'], bins=age_bins, labels=age_labels, right=False)
    age_group_sales = df.groupby('Age_Group', observed=False)['Sales'].mean().sort_values(ascending=False)
    best_age_group = age_group_sales.index[0]

    # Gender Analysis
    gender_sales = df.groupby('Customer_Gender', observed=False)['Sales'].mean()

    summary = f"""
    Advanced Sales Data Summary:

    Overall Sales Metrics:
    - Total Sales: ${total_sales:,.2f}
    - Average Sale: ${avg_sale:.2f}
    - Median Sale: ${median_sale:.2f}
    - Sales Standard Deviation: ${sales_std:.2f}

    Time-based Analysis:
    - Best Performing Month: {best_month}
    - Worst Performing Month: {worst_month}

    Product Analysis:
    - Top Selling Product (by value): {top_product}
    - Most Frequently Sold Product: {most_sold_product}

    Regional Performance:
    - Best Performing Region: {best_region}
    - Worst Performing Region: {worst_region}

    Customer Insights:
    - Average Customer Satisfaction: {avg_satisfaction:.2f}/5
    - Customer Satisfaction Standard Deviation: {satisfaction_std:.2f}
    - Best Performing Age Group: {best_age_group}
    - Gender-based Average Sales: Male=${gender_sales['Male']:.2f}, Female=${gender_sales['Female']:.2f}


    Key Observations:
    1. The sales data shows significant variability with a standard deviation of ${sales_std:.2f}.
    2. The {best_age_group} age group shows the highest average sales.
    3. Regional performance varies significantly, with {best_region} outperforming {worst_region}.
    4. The most valuable product ({top_product}) differs from the most frequently sold product ({most_sold_product}), suggesting potential for targeted marketing strategies.
    """

    return summary

In [9]:
advanced_summary = generate_advanced_data_summary(mydata)

In [10]:
advanced_summary

'\n    Advanced Sales Data Summary:\n\n    Overall Sales Metrics:\n    - Total Sales: $1,383,220.00\n    - Average Sale: $553.29\n    - Median Sale: $552.50\n    - Sales Standard Deviation: $260.10\n\n    Time-based Analysis:\n    - Best Performing Month: 2028-04\n    - Worst Performing Month: 2028-11\n\n    Product Analysis:\n    - Top Selling Product (by value): Widget A\n    - Most Frequently Sold Product: Widget A\n\n    Regional Performance:\n    - Best Performing Region: West\n    - Worst Performing Region: East\n\n    Customer Insights:\n    - Average Customer Satisfaction: 3.03/5\n    - Customer Satisfaction Standard Deviation: 1.16\n    - Best Performing Age Group: 18-25\n    - Gender-based Average Sales: Male=$547.56, Female=$558.96\n\n\n    Key Observations:\n    1. The sales data shows significant variability with a standard deviation of $260.10.\n    2. The 18-25 age group shows the highest average sales.\n    3. Regional performance varies significantly, with West outperf